In [3]:
# Print dataframe of results
from influxdb_client import InfluxDBClient
import pandas as pd

influxdb_url = "http://localhost:8086"
token = "random_token"
username = "influx-admin"
password = "ThisIsNotThePasswordYouAreLookingFor"
org = "ORG"
bucket = "system_state"
measurement = "base_result-cc2"
limit = 20

# Instantiate the QueryAPI
client = InfluxDBClient(url=influxdb_url, token=token, org=org, username=username, password=password)
query_api = client.query_api()

start_time = "2014-04-10T00:00:00Z"

# Construct the Flux query
query = f'''
from(bucket: "{bucket}")
    |> range(start: time(v: "{start_time}"))
    |> filter(fn: (r) => r["_measurement"] == "{measurement}")
    |> limit(n: {limit})
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

# Query the data
result_df = query_api.query_data_frame(query=query)


# If not empty
if not result_df.empty:
    # Convert 'T' to type integer
    result_df['T'] = result_df['T'].astype(int)

    # Rename the columns and display the result
    result_df.rename(columns={'_time': 'time', '_measurement': 'measurement'}, inplace=True)

    # Include 'AARE' and 'Thd' if they exist, else, ignore
    selected_columns = ['time', 'measurement', 'T', 'actual_value', 'predicted_value']
    selected_columns += ['AARE', 'Thd'] if 'AARE' in result_df.columns and 'Thd' in result_df.columns else []

    # If the result is not empty, select the columns we're interested in and rename them for clarity
    result_df = result_df[selected_columns]
else:
    print("No data found")

# Display the result
result_df


,time,measurement,T,actual_value,predicted_value,AARE,Thd
0,2014-04-10 00:04:00+00:00,base_result-cc2,0,91.958,0.000000,0.000000,0.000000
1,2014-04-10 00:09:00+00:00,base_result-cc2,1,94.798,0.000000,0.000000,0.000000
2,2014-04-10 00:14:00+00:00,base_result-cc2,2,92.208,0.000000,0.000000,0.000000
3,2014-04-10 00:19:00+00:00,base_result-cc2,3,93.722,93.617393,0.000000,0.000000
4,2014-04-10 00:24:00+00:00,base_result-cc2,4,93.042,93.947838,0.000000,0.000000
5,2014-04-10 00:29:00+00:00,base_result-cc2,5,92.958,93.406006,0.000000,0.000000
6,2014-04-10 00:34:00+00:00,base_result-cc2,6,95.708,93.210625,0.005224,0.000000
7,2014-04-10 00:39:00+00:00,base_result-cc2,7,95.250,94.897240,0.013550,0.000000
8,2014-04-10 00:44:00+00:00,base_result-cc2,8,94.458,94.897240,0.011539,0.020746
9,2014-04-10 00:49:00+00:00,base_result-cc2,9,92.750,94.897240,0.017649,0.025444
